Inspired by: https://realpython.com/simpy-simulating-with-python/

In [300]:
# !pip install simpy
import simpy
import numpy as np
import statistics

process_times = []


In [301]:
class Hospital(object):
    def __init__(self, env, num_nurses, num_radiologists, num_specialists):
        self.env = env
        self.nurse = simpy.Resource(env, num_nurses)
        self.radiologist = simpy.Resource(env, num_radiologists)
        self.specialist = simpy.Resource(env, num_specialists)

    def check_vitals(self, patient):
        yield self.env.timeout(np.random.randint(5, 15))
    
    def take_xray(self, patient):
        yield self.env.timeout(np.random.randint(15, 25))
    
    def consult_patient(self, patient):
        yield self.env.timeout(np.random.randint(10, 20))


In [302]:
def go_to_hospital(env, patient, hospital):
    # Patient arrives at the hospital
    arrival_time = env.now

    with hospital.nurse.request() as request:
        yield request
        print(f"Nurse starts checking vitals of patient {patient} at {env.now}")
        yield env.process(hospital.check_vitals(patient))
        print(f"Nurse finished checking patient {patient} at {env.now}")
    
    if np.random.choice([True, False], p=[.5, .5]):
        with hospital.radiologist.request() as request:
            yield request
            print(f"Radiologist takes xray of patient {patient} at {env.now}")
            yield env.process(hospital.take_xray(patient))
            print(f"Radiologist releases patient {patient} at {env.now}")

    with hospital.specialist.request() as request:
        yield request
        print(f"Specialist starts consult with patient {patient} at {env.now}")
        yield env.process(hospital.consult_patient(patient))
        print(f"Specialist finished consult with patient {patient} at {env.now}")

    # Patient goes home
    finish_time = env.now
    process_time = finish_time - arrival_time
    print(f">> Patient {patient} started at {arrival_time} and left at {finish_time} with total process time {process_time}")
    process_times.append(process_time)


In [303]:
def run_hospital(env, num_nurses, num_radiologists, num_specialists):
    hospital = Hospital(env, num_nurses, num_radiologists, num_specialists)
    
    for patient in range(3):
        env.process(go_to_hospital(env, patient, hospital))

    while True:
        yield env.timeout(0.20)  # Wait a bit before generating a new patient

        patient += 1
        env.process(go_to_hospital(env, patient, hospital))


In [304]:
def assess_process_quality(process_times):
    average_process = statistics.mean(process_times)
    num_patients = len(process_times)
    return average_process, num_patients


In [305]:
def main(params):
    # Setup
    np.random.seed(42)
    num_nurses, num_radiologists, num_specialists = params

    # Run the simulation
    env = simpy.Environment()
    env.process(run_hospital(env, num_nurses, num_radiologists, num_specialists))
    print("Running simulation...")
    env.run(until=90)

    # View the results
    average_process, num_patients = assess_process_quality(process_times)
    print(f"\nThe average process time is {average_process} minutes with {num_patients} patients handled.",)


In [306]:
params=[1,1,1]
main(params)


Running simulation...
Nurse starts checking vitals of patient 0 at 0
Nurse finished checking patient 0 at 11
Specialist starts consult with patient 0 at 11
Nurse starts checking vitals of patient 1 at 11
Nurse finished checking patient 1 at 20
Radiologist takes xray of patient 1 at 20
Nurse starts checking vitals of patient 2 at 20
Specialist finished consult with patient 0 at 28
>> Patient 0 started at 0 and left at 28 with total process time 28
Nurse finished checking patient 2 at 31
Nurse starts checking vitals of patient 3 at 31
Radiologist releases patient 1 at 37
Specialist starts consult with patient 1 at 37
Radiologist takes xray of patient 2 at 37
Nurse finished checking patient 3 at 43
Nurse starts checking vitals of patient 4 at 43
Nurse finished checking patient 4 at 50
Nurse starts checking vitals of patient 5 at 50
Specialist finished consult with patient 1 at 51
>> Patient 1 started at 0 and left at 51 with total process time 51
Radiologist releases patient 2 at 55
Speci